In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
# Snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.functions import pandas_udf


In [3]:
# Read credentials
with open('creds.json') as f:
    connection_parameters = json.load(f)    
session = Session.builder.configs(connection_parameters).create()

In [4]:
snowpark_version = VERSION
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Role                        : {}'.format(session.get_current_role()))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

Database                    : "HOUSING"
Schema                      : "PUBLIC"
Warehouse                   : "XSMALL_WH"
Role                        : "ACCOUNTADMIN"
Snowpark for Python version : 1.0.0


In [5]:
session.sql("SELECT count(*) FROM HOUSING.PUBLIC.HOUSINGPRICE").collect()

[Row(COUNT(*)=39570)]

In [6]:
housepricingdf = session.table("HOUSING.PUBLIC.HOUSINGPRICE")

In [7]:
housepricingdf=housepricingdf.to_pandas()
type(housepricingdf)

pandas.core.frame.DataFrame

In [8]:
housepricingdf.columns = map(lambda x: str(x).upper(), housepricingdf.columns)

In [9]:
housepricingdf.head()

,DATE,SUBURB,ADDRESS,ROOMS,TYPE,PRICE,METHOD,SELLERG,DATE.1,DISTANCE,...,BATHROOM,CAR,LANDSIZE,BUILDINGAREA,YEARBUILT,COUNCILAREA,LATTITUDE,LONGTITUDE,REGIONNAME,PROPERTYCOUNT
0,2016-01-28,Surrey Hills,999A Riversdale Rd,3,h,1205000.0,S,Fletchers,2016-01-28,11.2,...,1.0,2.0,490.0,NaN,NaN,Boroondara City Council,-37.8361,145.1006,Southern Metropolitan,5457.0
1,2016-01-28,Surrey Hills,1/10 Florence Rd,2,u,813000.0,S,Fletchers,2016-01-28,11.2,...,1.0,2.0,108.0,NaN,NaN,Boroondara City Council,-37.8276,145.1023,Southern Metropolitan,5457.0
2,2016-01-28,Canterbury,140 Canterbury Rd,4,h,NaN,SP,Fletchers,2016-01-28,9.0,...,2.0,0.0,808.0,198.0,1910.0,Boroondara City Council,-37.8235,145.0751,Southern Metropolitan,3265.0
3,2016-03-09,Williamstown,54 Twyford St,3,h,1535000.0,S,Greg,2016-03-09,8.0,...,1.0,1.0,507.0,186.0,1910.0,Hobsons Bay City Council,-37.8681,144.8994,Western Metropolitan,6380.0
4,2016-03-09,Newport,1/26 Thorpe St,3,h,826000.0,S,Village,2016-03-09,8.4,...,1.0,2.0,281.0,100.0,1957.0,Hobsons Bay City Council,-37.8457,144.8651,Western Metropolitan,5498.0


In [10]:
housepricingdf.isnull().sum()

DATE                 0
SUBURB               0
ADDRESS              0
ROOMS                0
TYPE                 0
PRICE             8649
METHOD               0
SELLERG              0
DATE.1               0
DISTANCE             2
POSTCODE             2
BEDROOM2          9617
BATHROOM          9629
CAR              10294
LANDSIZE         13554
BUILDINGAREA     24233
YEARBUILT        22295
COUNCILAREA          4
LATTITUDE         9352
LONGTITUDE        9352
REGIONNAME           4
PROPERTYCOUNT        4
dtype: int64

In [11]:
housepricingdf.nunique()

DATE                78
SUBURB             351
ADDRESS          34009
ROOMS               12
TYPE                 3
PRICE             2871
METHOD               9
SELLERG            388
DATE.1              78
DISTANCE           215
POSTCODE           211
BEDROOM2            15
BATHROOM            11
CAR                 15
LANDSIZE          1684
BUILDINGAREA       740
YEARBUILT          160
COUNCILAREA         33
LATTITUDE        13402
LONGTITUDE       14524
REGIONNAME           8
PROPERTYCOUNT      342
dtype: int64

In [12]:
cols=['SUBURB','ROOMS','TYPE','METHOD','SELLERG','REGIONNAME','PROPERTYCOUNT','DISTANCE','COUNCILAREA','BEDROOM2','BATHROOM'
           ,'CAR','LANDSIZE','BUILDINGAREA','PRICE']
housepricingdf=housepricingdf[cols]
housepricingdf

,SUBURB,ROOMS,TYPE,METHOD,SELLERG,REGIONNAME,PROPERTYCOUNT,DISTANCE,COUNCILAREA,BEDROOM2,BATHROOM,CAR,LANDSIZE,BUILDINGAREA,PRICE
0,Surrey Hills,3,h,S,Fletchers,Southern Metropolitan,5457.0,11.2,Boroondara City Council,3.0,1.0,2.0,490.0,NaN,1205000.0
1,Surrey Hills,2,u,S,Fletchers,Southern Metropolitan,5457.0,11.2,Boroondara City Council,2.0,1.0,2.0,108.0,NaN,813000.0
2,Canterbury,4,h,SP,Fletchers,Southern Metropolitan,3265.0,9.0,Boroondara City Council,4.0,2.0,0.0,808.0,198.0,NaN
3,Williamstown,3,h,S,Greg,Western Metropolitan,6380.0,8.0,Hobsons Bay City Council,3.0,1.0,1.0,507.0,186.0,1535000.0
4,Newport,3,h,S,Village,Western Metropolitan,5498.0,8.4,Hobsons Bay City Council,2.0,1.0,2.0,281.0,100.0,826000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39565,South Melbourne,2,u,SP,Dingle,Southern Metropolitan,5943.0,1.9,Port Phillip City Council,NaN,NaN,NaN,NaN,NaN,NaN
39566,South Kingsville,2,t,S,Village,Western Metropolitan,984.0,6.2,Hobsons Bay City Council,2.0,1.0,1.0,215.0,NaN,730000.0
39567,Roxburgh Park,4,h,S,Raine,Northern Metropolitan,5833.0,20.6,Hume City Council,4.0,2.0,1.0,530.0,0.0,622000.0
39568,Balwyn North,3,h,S,Jellis,Southern Metropolitan,7809.0,9.7,Boroondara City Council,3.0,2.0,2.0,863.0,NaN,2315000.0


In [13]:
housepricingdf.isnull().sum()

SUBURB               0
ROOMS                0
TYPE                 0
METHOD               0
SELLERG              0
REGIONNAME           4
PROPERTYCOUNT        4
DISTANCE             2
COUNCILAREA          4
BEDROOM2          9617
BATHROOM          9629
CAR              10294
LANDSIZE         13554
BUILDINGAREA     24233
PRICE             8649
dtype: int64

In [14]:
cols_zero = ['PROPERTYCOUNT','DISTANCE','BEDROOM2','BATHROOM','CAR']
housepricingdf[cols_zero]=housepricingdf[cols_zero].fillna(0)


In [15]:
housepricingdf.LANDSIZE=housepricingdf.LANDSIZE.fillna(housepricingdf.LANDSIZE.mean())
housepricingdf.BUILDINGAREA=housepricingdf.BUILDINGAREA.fillna(housepricingdf.BUILDINGAREA.mean())


In [16]:
housepricingdf.dropna(inplace=True)
housepricingdf.isnull().sum()

SUBURB           0
ROOMS            0
TYPE             0
METHOD           0
SELLERG          0
REGIONNAME       0
PROPERTYCOUNT    0
DISTANCE         0
COUNCILAREA      0
BEDROOM2         0
BATHROOM         0
CAR              0
LANDSIZE         0
BUILDINGAREA     0
PRICE            0
dtype: int64

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [18]:
# le.fit(housepricingdf) 
le1=LabelEncoder()
le2=LabelEncoder()
le3=LabelEncoder()
le4=LabelEncoder()
le5=LabelEncoder()
le6=LabelEncoder()
housepricingdf.SUBURB=le1.fit_transform(housepricingdf.SUBURB)
housepricingdf.TYPE = le2.fit_transform(housepricingdf.TYPE)
housepricingdf.METHOD = le3.fit_transform(housepricingdf.METHOD)
housepricingdf.SELLERG=le4.fit_transform(housepricingdf.SELLERG)
housepricingdf.REGIONNAME=le5.fit_transform(housepricingdf.REGIONNAME)
housepricingdf.COUNCILAREA = le6.fit_transform(housepricingdf.COUNCILAREA)
housepricingdf

,SUBURB,ROOMS,TYPE,METHOD,SELLERG,REGIONNAME,PROPERTYCOUNT,DISTANCE,COUNCILAREA,BEDROOM2,BATHROOM,CAR,LANDSIZE,BUILDINGAREA,PRICE
0,295,3,0,1,99,5,5457.0,11.2,2,3.0,1.0,2.0,490.0,162.042606,1205000.0
1,295,2,2,1,99,5,5457.0,11.2,2,2.0,1.0,2.0,108.0,162.042606,813000.0
3,333,3,0,1,115,6,6380.0,8.0,10,3.0,1.0,1.0,507.0,186.000000,1535000.0
4,231,3,0,1,312,6,5498.0,8.4,10,2.0,1.0,2.0,281.0,100.000000,826000.0
5,231,3,0,3,290,6,5498.0,8.4,10,3.0,2.0,2.0,301.0,195.000000,1025000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39563,280,4,0,1,118,2,7969.0,20.5,29,4.0,2.0,2.0,517.0,162.042606,774000.0
39566,278,2,1,1,312,6,984.0,6.2,10,2.0,1.0,1.0,215.0,162.042606,730000.0
39567,268,4,0,1,253,2,5833.0,20.6,11,4.0,2.0,1.0,530.0,0.000000,622000.0
39568,22,3,0,1,142,5,7809.0,9.7,2,3.0,2.0,2.0,863.0,162.042606,2315000.0


In [19]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor()

In [20]:
X3 = housepricingdf.drop(['PRICE'],axis=1)
y3=housepricingdf.PRICE

In [21]:
from sklearn.model_selection import train_test_split
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size = .20, random_state= 0)

In [22]:
knn.fit(X3_train, y3_train)
pred_y=knn.predict(X3_test)

In [23]:
from sklearn import metrics
# Calculated R Squared
print('R^2 =',metrics.explained_variance_score(y3_test,pred_y))

R^2 = 0.522422375222797


In [24]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, pred_y))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, pred_y))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, pred_y)))

Mean Absolute Error: 261232.9145860285
Mean Squared Error: 189502175479.44635
Root Mean Squared Error: 435318.47592245194


In [25]:
errors = abs(pred_y - y3_test)
MSE=round(np.mean(errors),2)
MAPE=100*(errors/y3_test)
accuracy=round (100 - np.mean(MAPE),2)
print(accuracy)

72.1


In [26]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors' : [3,4,5,6,7,10,15] ,    
              'weights' : ['uniform','distance'] ,
              'algorithm' : ['ball_tree', 'kd_tree', 'brute']}


grid_knn = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, refit=True, verbose=2, scoring = 'neg_mean_squared_error')

grid_knn.fit(X3_train, y3_train)


Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV] END algorithm=ball_tree, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=uniform; total time=   0.2s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=uniform; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=3, weights=distance; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=4, weights=uniform; total time=   0.1s
[CV] END algorithm=ball_tree, n_neighbors=

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [3, 4, 5, 6, 7, 10, 15],
                         'weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error', verbose=2)

In [27]:
best_score = np.sqrt(-grid_knn.best_score_)
print(best_score)    
print(grid_knn.best_params_)
print(grid_knn.best_estimator_)

425821.1433127453
{'algorithm': 'kd_tree', 'n_neighbors': 7, 'weights': 'distance'}
KNeighborsRegressor(algorithm='kd_tree', n_neighbors=7, weights='distance')


In [28]:
pred_knn = grid_knn.predict(X3_test)


In [29]:
from sklearn import metrics
# Calculated R Squared
print('R^2 =',metrics.explained_variance_score(y3_test,pred_knn))

R^2 = 0.5904944793939284


In [30]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y3_test, pred_knn))
print('Mean Squared Error:', metrics.mean_squared_error(y3_test, pred_knn))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y3_test, pred_knn)))

Mean Absolute Error: 217378.8039679905
Mean Squared Error: 162474661315.89264
Root Mean Squared Error: 403081.4574200761


In [31]:
errors = abs(pred_knn - y3_test)
MSE=round(np.mean(errors),2)
MAPE=100*(errors/y3_test)
accuracy=round (100 - np.mean(MAPE),2)
print(accuracy)

76.52


In [32]:
df3= pd.DataFrame({'Actual_Price': np.round(y3_test), 
                   'Predicted_Price': np.round(pred_knn),
                 })
df3['difference']=df3.apply(lambda x: x.Predicted_Price-x.Actual_Price,axis=1)
df3.head(10)

,Actual_Price,Predicted_Price,difference
10111,440000.0,724228.0,284228.0
30066,850250.0,1056014.0,205764.0
17936,540000.0,538180.0,-1820.0
31791,920000.0,920000.0,0.0
35810,512500.0,512500.0,0.0
341,620000.0,652007.0,32007.0
17511,735000.0,695000.0,-40000.0
31757,1040000.0,1093959.0,53959.0
5629,365000.0,591558.0,226558.0
23725,620000.0,502407.0,-117593.0


In [33]:

# Creating a User Defined Function within Snowflake to do the scoring there
def predict_pandas_udf_KNR(df3: pd.DataFrame) -> pd.Series:
    return pd.Series(grid_knn.predict(df3))  


In [34]:
housepricingdf.columns = map(lambda x: str(x).upper(), housepricingdf.columns)
snowdf_details = session.createDataFrame(housepricingdf)
snowdf_details.show()
snowdf_details.write.mode("overwrite").saveAsTable("HOUSING.PUBLIC.FULL_HOUSINGPRICE_encoded_KNR") 

session.table("HOUSING.PUBLIC.FULL_HOUSINGPRICE_encoded_KNR").show(5)

create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUBURB"  |"ROOMS"  |"TYPE"  |"METHOD"  |"SELLERG"  |"REGIONNAME"  |"PROPERTYCOUNT"  |"DISTANCE"  |"COUNCILAREA"  |"BEDROOM2"  |"BATHROOM"  |"CAR"  |"LANDSIZE"  |"BUILDINGAREA"      |"PRICE"    |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|295       |3        |0       |1         |99         |5             |5457.0           |11.2        |2              |3.0         |1.0         |2.0    |490.0       |162.04260635717546  |1205000.0  |
|295       |2        |2       |1         |99         |5             |5457.0           |11.2        |2              |2.0         |1.0         |2.0    |108.0       |162.04260635717546  |813000.0   |
|333       |3  

In [35]:
KNR_model_vec = pandas_udf(func=predict_pandas_udf_KNR,
                                return_type=FloatType(),
                                input_types=[IntegerType(),IntegerType(),IntegerType(),IntegerType(),IntegerType(),IntegerType(),FloatType(),FloatType(),FloatType(),IntegerType(),FloatType(),FloatType(),FloatType(),FloatType()],
                                session=session,
                                packages = ("pandas","scikit-learn"), max_batch_size=200)

In [36]:

# Calling the UDF to do the scoring (pushing down to Snowflake)
output_KNR = session.table('HOUSING.PUBLIC.FULL_HOUSINGPRICE_encoded_KNR').select(*list(X3.columns),
                    KNR_model_vec(list(X3.columns)).alias('PREDICTED_PRICE'),
                    (F.col('Price')).alias('ACTUAL_PRICE')                                              
                    )

output_KNR.show(5)


Passing arguments to a UDF with a list or tuple is deprecated. We still respect this invocation but please consider passing variable-length arguments without a list or tuple.


-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUBURB"  |"ROOMS"  |"TYPE"  |"METHOD"  |"SELLERG"  |"REGIONNAME"  |"PROPERTYCOUNT"  |"DISTANCE"  |"COUNCILAREA"  |"BEDROOM2"  |"BATHROOM"  |"CAR"  |"LANDSIZE"  |"BUILDINGAREA"      |"PREDICTED_PRICE"  |"ACTUAL_PRICE"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|295       |3        |0       |1         |99         |5             |5457.0           |11.2        |2              |3.0         |1.0         |2.0    |490.0       |162.04260635717546  |1205000.0          |1205000.0       |
|295       |2        |2       |1         |99         |5             |5457.0           |11.2        |2           

In [37]:
output_KNR=output_KNR.to_pandas()
output_KNR.head()

,SUBURB,ROOMS,TYPE,METHOD,SELLERG,REGIONNAME,PROPERTYCOUNT,DISTANCE,COUNCILAREA,BEDROOM2,BATHROOM,CAR,LANDSIZE,BUILDINGAREA,PREDICTED_PRICE,ACTUAL_PRICE
0,333,3,0,1,115,6,6380.0,8.0,10,3.0,1.0,1.0,507.000000,186.000000,1.535000e+06,1535000.0
1,218,4,0,1,142,0,2079.0,11.8,2,4.0,2.0,2.0,1388.000000,162.042606,2.850000e+06,2850000.0
2,40,2,0,1,22,6,3589.0,10.8,16,2.0,1.0,1.0,566.000000,75.000000,7.823876e+05,645000.0
3,30,3,1,1,48,5,10969.0,13.9,8,0.0,0.0,0.0,600.989891,162.042606,1.021000e+06,990000.0
4,239,2,0,1,22,2,2651.0,11.5,24,2.0,1.0,2.0,577.000000,162.042606,5.920000e+05,592000.0


In [38]:
output_KNR.SUBURB=le1.inverse_transform(output_KNR.SUBURB)
output_KNR.TYPE = le2.inverse_transform(output_KNR.TYPE)
output_KNR.METHOD = le3.inverse_transform(output_KNR.METHOD)
output_KNR.SELLERG=le4.inverse_transform(output_KNR.SELLERG)
output_KNR.REGIONNAME=le5.inverse_transform(output_KNR.REGIONNAME)
output_KNR.COUNCILAREA = le6.inverse_transform(output_KNR.COUNCILAREA)


In [39]:
output_KNR.head()

,SUBURB,ROOMS,TYPE,METHOD,SELLERG,REGIONNAME,PROPERTYCOUNT,DISTANCE,COUNCILAREA,BEDROOM2,BATHROOM,CAR,LANDSIZE,BUILDINGAREA,PREDICTED_PRICE,ACTUAL_PRICE
0,Williamstown,3,h,S,Greg,Western Metropolitan,6380.0,8.0,Hobsons Bay City Council,3.0,1.0,1.0,507.000000,186.000000,1.535000e+06,1535000.0
1,Mont Albert,4,h,S,Jellis,Eastern Metropolitan,2079.0,11.8,Boroondara City Council,4.0,2.0,2.0,1388.000000,162.042606,2.850000e+06,2850000.0
2,Braybrook,2,h,S,Barry,Western Metropolitan,3589.0,10.8,Maribyrnong City Council,2.0,1.0,1.0,566.000000,75.000000,7.823876e+05,645000.0
3,Bentleigh East,3,t,S,Buxton,Southern Metropolitan,10969.0,13.9,Glen Eira City Council,0.0,0.0,0.0,600.989891,162.042606,1.021000e+06,990000.0
4,Oak Park,2,h,S,Barry,Northern Metropolitan,2651.0,11.5,Moreland City Council,2.0,1.0,2.0,577.000000,162.042606,5.920000e+05,592000.0


In [40]:
snowdf_details_KNR = session.createDataFrame(output_KNR)
snowdf_details_KNR.show()
snowdf_details_KNR.write.mode("overwrite").saveAsTable("HOUSING.PUBLIC.FULL_HOUSINGPRICE_PREDICTED_KNR") 




-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUBURB"        |"ROOMS"  |"TYPE"  |"METHOD"  |"SELLERG"      |"REGIONNAME"           |"PROPERTYCOUNT"  |"DISTANCE"  |"COUNCILAREA"             |"BEDROOM2"  |"BATHROOM"  |"CAR"  |"LANDSIZE"         |"BUILDINGAREA"      |"PREDICTED_PRICE"   |"ACTUAL_PRICE"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Williamstown    |3        |h       |S         |Greg           |Western Metropolitan   |6380.0           |8.0         |Hobsons Bay City Council  |3.0         |1.0         |1.0    |507.0              |186.0               

In [41]:
# Also get a local dataframe to review the results
snowdf_details_KNR=snowdf_details_KNR.toPandas()
snowdf_details_KNR

,SUBURB,ROOMS,TYPE,METHOD,SELLERG,REGIONNAME,PROPERTYCOUNT,DISTANCE,COUNCILAREA,BEDROOM2,BATHROOM,CAR,LANDSIZE,BUILDINGAREA,PREDICTED_PRICE,ACTUAL_PRICE
0,Williamstown,3,h,S,Greg,Western Metropolitan,6380.0,8.0,Hobsons Bay City Council,3.0,1.0,1.0,507.000000,186.000000,1.535000e+06,1535000.0
1,Mont Albert,4,h,S,Jellis,Eastern Metropolitan,2079.0,11.8,Boroondara City Council,4.0,2.0,2.0,1388.000000,162.042606,2.850000e+06,2850000.0
2,Braybrook,2,h,S,Barry,Western Metropolitan,3589.0,10.8,Maribyrnong City Council,2.0,1.0,1.0,566.000000,75.000000,7.823876e+05,645000.0
3,Bentleigh East,3,t,S,Buxton,Southern Metropolitan,10969.0,13.9,Glen Eira City Council,0.0,0.0,0.0,600.989891,162.042606,1.021000e+06,990000.0
4,Oak Park,2,h,S,Barry,Northern Metropolitan,2651.0,11.5,Moreland City Council,2.0,1.0,2.0,577.000000,162.042606,5.920000e+05,592000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30912,Brighton,3,t,VB,Jellis,Southern Metropolitan,10579.0,10.5,Bayside City Council,3.0,2.0,2.0,370.000000,162.042606,1.900000e+06,1900000.0
30913,Reservoir,2,u,SP,Ray,Northern Metropolitan,21650.0,12.0,Darebin City Council,2.0,1.0,2.0,176.000000,90.000000,4.600000e+05,460000.0
30914,Parkdale,2,u,S,Barry,South-Eastern Metropolitan,5087.0,21.5,Kingston City Council,0.0,0.0,0.0,600.989891,162.042606,6.500000e+05,650000.0
30915,Thomastown,4,h,SP,Skad,Northern Metropolitan,7955.0,15.3,Whittlesea City Council,4.0,1.0,4.0,567.000000,162.042606,5.925000e+05,592500.0
